# House Building 3

## より複雑な問題

* 5件の家を、JoeとJimの2人で建てる  
* 工程毎に担当者が決まっている
* 別の家に移る場合は遷移時間がかかる
* 家毎にReleaseDate以降に着手する必要がある
* 家毎にDueDateが決まっている。遅れてもいいが、Weightで規定されるコストが余分にかかる
* 評価関数は「遅延によるコスト」+「家毎の建築期間」でこれを最小化する

元リンク

https://ibmdecisionoptimization.github.io/tutorials/html/Scheduling_Tutorial.html

API reference

https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html

In [ ]:
# 家の件数
NbHouses = 5

# 作業者
WorkerNames = ["Joe", "Jim"]

# 工程名
TaskNames = ["masonry", "carpentry", "plumbing", 
             "ceiling", "roofing", "painting", 
             "windows", "facade", "garden", "moving"]

# 工程毎の期間
Duration =  [35, 15, 40, 15, 5, 10, 5, 10, 5, 5]

# 工程毎の作業者分担
Worker = {"masonry"  : "Joe" , 
          "carpentry": "Joe" , 
          "plumbing" : "Jim" , 
          "ceiling"  : "Jim" , 
          "roofing"  : "Joe" , 
          "painting" : "Jim" , 
          "windows"  : "Jim" , 
          "facade"   : "Joe" , 
          "garden"   : "Joe" , 
          "moving"   : "Jim"}

# 着手日 (必ず守る必要あり)
ReleaseDate = [  0,     0,   151,    59,   243]

# 終了日 (遅れていいがコストが発生)
DueDate     = [120,   212,   304,   181,   425]

# 遅延コスト
Weight      = [100.0, 100.0, 100.0, 200.0, 100.0]

# 工程間依存関係
Precedences = [("masonry", "carpentry"),("masonry", "plumbing"),
               ("masonry", "ceiling"), ("carpentry", "roofing"),
               ("ceiling", "painting"), ("roofing", "windows"),  
               ("roofing", "facade"), ("plumbing", "facade"),
               ("roofing", "garden"), ("plumbing", "garden"),
               ("windows", "moving"), ("facade", "moving"),  
               ("garden", "moving"), ("painting", "moving")]

# 家IDのリスト([0, 1, 2, 3, 4])を生成
Houses = range(NbHouses)

In [ ]:
# ライブラリのインポートとモデルインスタンスの生成

import sys
from docplex.cp.model import *
mdl2 = CpoModel()

In [ ]:
# housesは家毎のInterval変数の配列
# 開始日はReleaseDateにより個別に指定

houses = [mdl2.interval_var(start=(ReleaseDate[i], INTERVAL_MAX), name="house"+str(i)) for i in Houses]

In [ ]:
# <house_id>_<タスク名>でタスクごとのラベルを振る
# 辞書 TaskNames_ids はタスク毎のラベルからタスクIDを取得するための辞書
# itvsは、(house_id, task_id)から該当するInterval変数を取得するための辞書
# タスク毎の期間(size)はDurationで決められている値を初期設定する

TaskNames_ids = {}
itvs = {}
for h in Houses:
    for i,t in enumerate(TaskNames):
        _name = str(h)+"_"+str(t)
        itvs[(h,t)] = mdl2.interval_var(size=Duration[i], name=_name)
        TaskNames_ids[_name] = i

In [ ]:
# itvsの辞書の内容確認

print(itvs[(3, 'masonry')])

In [ ]:
# タスク間の依存関係定義

for h in Houses:
    for p in Precedences:
        mdl2.add(mdl2.end_before_start(itvs[(h,p[0])], itvs[(h,p[1])]) )

In [ ]:
# span制約の設定

# 家のInterval変数は、個別タスクの集計で定まる。その関係をspan制約で設定
for h in Houses:
    mdl2.add( mdl2.span(houses[h], [itvs[(h,t)] for t in TaskNames] ) )

In [ ]:
# 遷移時間の設定　

# 複数のタスク間の遷移時間を設定する
# (タスク間で対象の家を変える場合のコスト定義)
transitionTimes = transition_matrix(NbHouses)
for i in Houses:
    for j in Houses:
        transitionTimes.set_value(i, j, int(abs(i - j)))

In [ ]:
# workersの定義

# 作業者(Joe, Jim)ごとにシーケンス変数として定義
workers = {w : mdl2.sequence_var([ itvs[(h,t)] for h in Houses for t in TaskNames if Worker[t]==w ], 
                                types=[h for h in Houses for t in TaskNames if Worker[t]==w ], name="workers_"+w)   
           for w in WorkerNames}

In [ ]:
print(workers['Jim'])

In [ ]:
# 作業者は同時に一つのタスクしかできない
# タスク間で家を変更する場合は遷移時間が必要

for w in WorkerNames:
    mdl2.add( mdl2.no_overlap(workers[w], transitionTimes) )

In [ ]:
# 目的関数の定義
# 「遅延コスト」+「建築期間」を家毎に集計した値を最小化する

mdl2.add( 
    mdl2.minimize( 
        mdl2.sum(Weight[h] * mdl2.max([0, mdl2.end_of(houses[h])-DueDate[h]]) +\
                 mdl2.length_of(houses[h]) for h in Houses) 
    ) 
)

In [ ]:
# モデルを解く

print("\nSolving model....")
msol2 = mdl2.solve(FailLimit=30000)
print("done")

In [ ]:
# 目的関数値の確認

print("Cost will be " + str(msol2.get_objective_values()[0]))

In [ ]:
# 結果のグラフ表示
# グラフをダブルクリックすると詳細表示になる

import docplex.cp.utils_visu as visu
import matplotlib.pyplot as plt
%matplotlib inline
#Change the plot size
from pylab import rcParams
rcParams['figure.figsize'] = 50,  3

def showsequence(msol, s, setup, tp):
    seq = msol.get_var_solution(s)
    visu.sequence(name=s.get_name())
    vs = seq.get_value()
    for v in vs:
        nm = v.get_name()
        visu.interval(v, tp[TaskNames_ids[nm]], nm)
    for i in range(len(vs) - 1):
        end = vs[i].get_end()
        tp1 = tp[TaskNames_ids[vs[i].get_name()]]
        tp2 = tp[TaskNames_ids[vs[i + 1].get_name()]]
        visu.transition(end, end + setup.get_value(tp1, tp2))

visu.timeline("Solution for SchedSetup")
for w in WorkerNames:
    types=[h for h in Houses for t in TaskNames if Worker[t]==w]
    showsequence(msol2, workers[w], transitionTimes, types)
visu.show()